# Classification using BERT embeddings

## Loading data

In [1]:
import pandas as pd

# The train and test data should have a column named cat1, containing the labels of the data
# The corresponding embeddings should be in this folder
# train_data = pd.read_csv('../data/formatted/train_data.csv')
# test_data = pd.read_csv('../data/formatted/test_data.csv')

train_data = pd.read_csv('../../Datasets/train_file_1M_cat1.csv')
test_data = pd.read_csv('../../Datasets/test_file_1M_cat1.csv')

train_data_labels = list(train_data['cat1'])
test_data_labels = list(test_data['cat1'])

In [2]:
# Reading the train data embeddings
training_data_embeddings = []
with open('../../8-bert_service/train_vecs_untoken_1M') as vecs_file:
    for index, line in enumerate(vecs_file):
        training_data_embeddings.append(list(map(float, line.split())))

In [3]:
# Reading the test data embeddings
test_data_embeddings = []
with open('../../8-bert_service/test_vecs_untoken_1M') as vecs_file:
    for index, line in enumerate(vecs_file):
        test_data_embeddings.append(list(map(float, line.split())))

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lm = LogisticRegression(solver='lbfgs',max_iter=10000, multi_class='multinomial', verbose=2)
lm.fit(training_data_embeddings, train_data_labels)

In [11]:
from sklearn.metrics import classification_report

prediction_labels = lm.predict(test_data_embeddings)
print(classification_report(list(test_data_labels), list(prediction_labels)))

              precision    recall  f1-score   support

           1       0.81      0.80      0.81     21674
           2       0.63      0.72      0.67     28604
          12       0.74      0.70      0.72     20714
          38       0.72      0.65      0.68     16511
          67       0.81      0.83      0.82     31741
          79       0.49      0.27      0.34     11070
         125       0.78      0.88      0.83     21342
         143       0.96      0.97      0.97     46593
         151       0.51      0.19      0.27       472
         191       0.81      0.75      0.78      1279

   micro avg       0.79      0.79      0.79    200000
   macro avg       0.72      0.68      0.69    200000
weighted avg       0.78      0.79      0.78    200000



## XGBoost

In [ ]:
import xgboost as xgb
import numpy as np

### Training an XGBoost Classifier using BERT embeddings

In [ ]:
clf = xgb.XGBClassifier(
    max_depth=7,
    n_estimators=100,
    objective='multi:softmax',
    n_jobs=30,
    silent=False,
    num_class=10
)

In [ ]:
clf.fit(np.array(training_data_embeddings), np.array(train_data_labels))

In [ ]:
clf.save_model('xgb_model')

### Testing the embeddings

In [ ]:
# Predicting
prediction_labels = clf.predict(test_data_embeddings)

In [ ]:
from sklearn.metrics import classification_report
classification_report(test_data_labels, list(prediction_labels))